In [1]:
import sys
sys.path.append('../')
import pandas as pd
from utils.classify_ads import *
from utils.df_utils import set_week_and_year
from plataforms.utils.calculate import *

# Procesamiento de datos
from plataforms.utils.process_data import get_value_counts_per_brand

# Obtener los datos generales de las publicaciones através de la BD de mongodb
from utils.mongodb_call import get_data_general

## Meta

In [2]:
# Lectura del archivo de Meta
df_meta_ads = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\BD Ads\Abril\MX BD Ads saturación mensual - Meta 08042025.csv")

# Aplicar la función a la columna Ad name
df_meta_ads['Marca'] = df_meta_ads['Ad name'].apply(classify_ads)

# Agregar columnas con el número de semana y año
df_meta_ads = set_week_and_year(df_meta_ads)

## Google Ads

In [3]:
# Lectura del archivo de Google Ads
df_google_ads = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\BD Ads\Abril\MX BD Ads saturación mensual - GAds 08042025.csv")

# Aplicar la función a la columna Ad name
df_google_ads['Marca'] = df_google_ads['Ad group name'].apply(classify_ads)

# Agregar columnas con el número de semana y año
df_google_ads = set_week_and_year(df_google_ads)

## Calcular el dinero de cada marca por plataforma

In [4]:
# Calculo de inversión por semana y año de ambas plataformas
df_meta_ads = calculate_invest_per_week(df_meta_ads)
df_google_ads = calculate_invest_per_week(df_google_ads)

In [5]:
invest_total = calculate_invest_from_plataforms(df_meta_ads, df_google_ads, show_result = True)

,Marca,Semana,Año,Cost
29,Generica,1,2025,97361.79
0,Bajaj,1,2025,58725.58
44,Honda,1,2025,30336.70
73,MB,1,2025,12291.64
109,Vento,1,2025,10027.66
...,...,...,...,...
72,Italika,15,2025,1522.79
106,TVS,15,2025,1340.37
123,Vento,15,2025,827.06
28,Dinamo,15,2025,118.02


## Distribuir generica entre las marcas

Acá queremos saber como se distribuye el dinero invertido en cada una de las marcas. Para eso, se tiene que saber cuales son los modelos que tenemos listados en el marketplace y distribuirlo

In [6]:
# Data con la información de los modelos en las primeras 18 posiciones
df_data_mx = get_data_general()

In [7]:
# Cantidad de modelos por marca listados en la primera página del marketplace
df_counts_per_brand, total_models = get_value_counts_per_brand(df_data_mx)
df_counts_per_brand

,brand,count
0,Bajaj,11
1,Vento,4
2,Honda,1
3,Italika,1
4,TVS,1


In [8]:
# Inversión de la genérica en la semana 14
generic_invest = calculate_generic_invest(invest_total, 14)

df_total_distribution = calculate_generic_invest_distribution(df_counts_per_brand, generic_invest, total_models, show_result = True)

Inversión genérica semana 14 es: 157999


,brand,count,Cost
0,Bajaj,11,96554.94
1,Vento,4,35110.89
2,Honda,1,8777.72
3,Italika,1,8777.72
4,TVS,1,8777.72
5,Total,18,157999.00


In [9]:
df_distribute_invest = distribute_generic_invest(invest_total, 
                                                 14, 
                                                 df_total_distribution, 
                                                 show_result = True)

c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\plataforms\utils\calculate_utils.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['brand_categoria'] = df['brand'].apply(categorizar_marca)


,brand_categoria,original_cost,distribute_cost,total_cost
0,Bajaj,141960.59,96554.94,238515.53
1,Otros,54105.42,0.00,54105.42
2,Honda,32090.55,8777.72,40868.27
3,Vento,4460.49,35110.89,39571.38
4,TVS,9753.55,8777.72,18531.27
5,Italika,7392.34,8777.72,16170.06
6,Total,249762.94,157998.99,407761.93


In [10]:
df_distribute_invest

,brand_categoria,original_cost,distribute_cost,total_cost
0,Bajaj,141960.59,96554.94,238515.53
1,Otros,54105.42,0.00,54105.42
2,Honda,32090.55,8777.72,40868.27
3,Vento,4460.49,35110.89,39571.38
4,TVS,9753.55,8777.72,18531.27
5,Italika,7392.34,8777.72,16170.06
6,Total,249762.94,157998.99,407761.93
